In [1]:
 # !pip install --upgrade pip
# !pip install pytorch
# !pip install transformers
# ! sudo apt install tesseract-ocr
# ! pip install pytesseract\n",
# ! pip install transformers datasets

In [2]:
import json
import os
import re
import pickle
from tqdm import tqdm
import random


from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer #tokanization and removel of punctuation \n",

from transformers import BertModel
from transformers import BertTokenizer
import torch

import pandas as pd
import numpy as np
np.random.seed(42)
import tensorflow as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

# from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Layer#, InputSpec
from tensorflow.keras import initializers, regularizers, constraints
#from tensorflow.keras.callbacks import Callback
from tensorflow.keras.layers import Input, Dense, GRU, Bidirectional, TimeDistributed, Dropout
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model

#from sklearn.metrics import roc_auc_score

2022-04-05 12:04:56.181133: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-05 12:04:56.181161: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
#classes
txt_path='/media/umar_visionx/Backup Plus/Active/Faizan/doc_classifier_classes.txt'
with open(txt_path,'r') as file:
    classes=file.read().split('\n')
print(classes)

['Admin Note', 'Clinical History And Summary', 'Consult Note', 'Cover Page', 'Imaging Note', 'Insurance Authorization', 'Intake Forms', 'Lab Test', 'Other', 'Patient Profile', 'Prescriptions', 'Referral Letter', 'Requisition Form']


In [3]:
ocr_train='/media/umar_visionx/Backup Plus/Active/Faizan/data_copy/ocr'
ocr_test='/media/umar_visionx/Backup Plus/Active/Faizan/data_copy/test-set/ocr_textract'
os.listdir(ocr_train)[0:5]

['North_Texas_Corp_LoneStar_2020-11-24_0012169206217_2020-11-24-23.36.36_OK_HSA0.png.ocr.json',
 'North_Texas_Amarillo_2020-11-24_0018479841164_2020-11-24-21.45.11_OK_7NV1.png.ocr.json',
 '003B35A1_0.png.ocr.json',
 'Mid_Florida_Kissimmee_West_2020-11-25_0016173936905_2020-11-25-10.12.32_OK_BMS0.pkl',
 'North_Texas_Amarillo_2020-11-25_0014693513968_2020-11-25-14.40.47_OK_A7T1.pkl']

In [5]:
#making folders and saving the text files (with extracted text from image) in related folders
def extract_text(source_path,dest_path,ocr_path):
        #first making folders
        try:
            os.makedirs(dest_path)
            for ent in classes:
                folder_path=os.path.join(dest_path,ent)
                os.makedirs(folder_path)
            print("Folder making process completed")
        except:
            pass
        for folder in os.listdir(source_path):
            folder_path=os.path.join(source_path,folder)
            for files in os.listdir(folder_path):
                    ocr_file=os.path.splitext(files)[0] + ".png.ocr.json"#making the name of ocr file to get access from ocr folder
                    ocr_f_path=os.path.join(ocr_path,ocr_file)#making directory of ocr_file in ocr folder
                    try:
                        with open(ocr_f_path,'r') as f:
                            output=json.load(f)
                        
                        if isinstance(output,dict):#if type of 'output' is dictionary
                            words=[]
                            for block in output['Blocks']:
                                if block['BlockType']=='WORD':
                                    words.append(block["Text"])
                            para= ' '.join(words)
                            
                        elif isinstance(output,list):#if type of 'output' is list
                            output = output[0]
                            para = " ".join([i["Text"] for i in output["Blocks"] if i["BlockType"] == "LINE"])
                        
                        elif isinstance(output,tuple):
                            output = output[0]
                            para = " ".join([i["Text"] for i in output["Blocks"] if i["BlockType"] == "LINE"])
                        
                        #making directory to save the files with same file and folder name
                        txt_file=os.path.splitext(files)[0] + ".txt" #making same name of file as image
                        txt_path=os.path.join(dest_path,os.path.join(folder,txt_file))
                        #writing and saving file
                        with open(txt_path, "w") as file:
                            file.write(para)
                    except:
                        pass
                    
        print("All text files are created successfully")




In [2]:
#extracting text from images and creating text files \n",
train_image="/media/umar_visionx/Backup Plus/Active/Faizan/image_data"#image data source path
test_image="/media/umar_visionx/Backup Plus/Active/Faizan/test_images"#test image data source path
train_text="/media/umar_visionx/Backup Plus/Active/Faizan/textrect_dataset"#extracted text files destinatio path for image data
test_text="/media/umar_visionx/Backup Plus/Active/Faizan/textrect_test_dataset"#extracted text files destination path from test image data
# extract_text(train_image,train_text,ocr_train)
# extract_text(test_image,test_text,ocr_test)

In [7]:
#total number of json files in ocr folder
lst=[]
for files in os.listdir(ocr_train):
    if files.endswith('.json'):
        lst.append(files)
print(len(lst))

18678


In [8]:
#getting the sum of files in each folder\n",
def sum_it(path):
    total=[]
    for folder in os.listdir(path):
        folder_path=os.path.join(path,folder)
        total.append(len(os.listdir(folder_path)))
    print(sum(total))
sum_it(train_image)
sum_it(test_image)
sum_it(train_text)
sum_it(test_text)

11660
1810
11660
1670


In [9]:
#removing special character and numbers
def rem_sp_char(words):
    pattern = r'[^a-zA-z0-9.,!?/:;\"\'\s]'  # defined pattern to keep
    words=re.sub(pattern, '', words)
    return words

#tokanizing and removing punctuations
def remove_punct(words):
  tokenizer = RegexpTokenizer(r'\w+')
  tokens=tokenizer.tokenize(words)
  return tokens

#lemmitization
def lemmitizar(words):
  lemmatized_words=[]
  l = WordNetLemmatizer()
  for word in words:
      token = l.lemmatize(word)
      lemmatized_words.append(token)
  return lemmatized_words

#removing stop-words
def remove_stopwords(words):
  filter_words = []
  Stopwords = stopwords.words('english')
  for word in words:
      if word not in Stopwords:
          filter_words.append(word)
  return filter_words

#converting to lowercase
def lower_case(words):
  lower_words=[]
  for word in words:
    lower_words.append(word.lower())
  return lower_words

In [3]:
#label encoding
labl=[]
for folder in os.listdir(train_text):
  labl.append(folder)#making the list of folder names which are labels


#encoding the labels from the list
label_index={}
for index,label in enumerate(labl):
  label_index[label]=index

label_index#label to index

{'Admin Note': 0,
 'Clinical History And Summary': 1,
 'Consult Note': 2,
 'Cover Page': 3,
 'Imaging Note': 4,
 'Insurance Authorization': 5,
 'Intake Forms': 6,
 'Lab Test': 7,
 'Other': 8,
 'Patient Profile': 9,
 'Prescriptions': 10,
 'Referral Letter': 11,
 'Requisition Form': 12}

In [11]:
#making dictionaries and lists of text and labels
def preprocessed_df(data_path):
        #txt_file_path=[]
        labels=[]
        text=[]
        encoded_labels=[]
        for clss in os.listdir(data_path):
            clss_path=os.path.join(data_path,clss)
            for txt_file in os.listdir(clss_path):
                file_path=os.path.join(clss_path,txt_file)
                #reading text file
                with open(file_path, "r") as file:
                    para = file.read().rstrip('\n')#rstrip method removes trailing character based on string argument passed
                #apply preprocessing functions on para
                words = rem_sp_char(para) #removing special character and numbers
                tokens = remove_punct(words) #removing punctuations and tokanization
                tokens = lemmitizar(tokens) #lemmitization
                tokens = remove_stopwords(tokens) ##removing stop-words
                tokens = lower_case(tokens) #converting to lowercase
    
                #joining words with space and removing single tetter tokens
                parag=' '.join(tokens)

                #inseting data into lists
                text.append(parag)
                #text_file_path.append(file_path)
                labels.append(clss)
    #label encoding
        for ent in labels:
            encoded_labels.append(label_index[ent])
        
    
    
        df=pd.DataFrame()
        df['text']=text
        df['labels']=encoded_labels
        return df

In [12]:
df=preprocessed_df(train_text)
df_test=preprocessed_df(test_text)
df.head()

,text,labels
0,7 29 2021 10 42 alberta health services rrduse...,0
1,7 30 2021 09 34 alberta health services rrduse...,0
2,7 29 2021 11 03 alberta health services rrduse...,0
3,8 4 2021 08 25 alber ta health services rrduse...,0
4,ahs 8 4 2021 7 23 05 am page 2 002 fax server ...,0


In [13]:
print(len(df))
print(len(df_test))

11660
1670


In [14]:
#BERT base model
# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased',output_hidden_states = True,)#model returns all hidden-states.
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [15]:
from datasets import Dataset
train_dataset = Dataset.from_pandas(df)
#val_dataset = Dataset.from_pandas(df_val)
test_dataset = Dataset.from_pandas(df_test)

In [16]:
def encode_example(sample,max_seq_length=512):
    words=sample['text']
    label=sample['labels']
    #splitting all words of sentence in to into tokens
    tokenized_text = tokenizer.tokenize(words)
    # Truncation of token_boxes
    special_tokens_count = 2 
    if len(tokenized_text) > max_seq_length - special_tokens_count:
      tokenized_text = tokenized_text[: (max_seq_length - special_tokens_count)]#to make place available for cls and sep tokens
    # Adding CLS and SEP tokens
    input_sequence = ["[CLS]"] + tokenized_text + ["[SEP]"]
    # Map the tokens to their vocabulary indeces
    indexed_tokens = tokenizer.convert_tokens_to_ids(input_sequence)
    #padding length
    padding_length = max_seq_length - len(input_sequence)
    indexed_tokens += [0] * padding_length#zero padding to maximum length
    #as we are giving as a single sentence so mark each token as belonging to sentence 1
    segments_ids = [1] * max_seq_length
    pad_masks = [1] * len(input_sequence) + [0] * padding_length


    assert len(indexed_tokens) == max_seq_length
    assert len(segments_ids) ==max_seq_length
    assert len(pad_masks) == max_seq_length

    #converting into pytorch tensors
    tokens_tensor = torch.tensor(indexed_tokens)
    segments_tensors = torch.tensor(segments_ids)
    pad_mask_tensors=torch.tensor(pad_masks)
    label_tensors=torch.tensor(label)
    
  
    tok_tensors.append(tokens_tensor)
    seg_tensors.append(segments_tensors)
    lab_tensors.append(label_tensors)
    pad_msk_tensors.append(pad_mask_tensors)

In [17]:
def iter_df(dataset):
    global tok_tensors,seg_tensors,lab_tensors,pad_msk_tensors

    tok_tensors=[]
    seg_tensors=[]
    lab_tensors=[]
    pad_msk_tensors=[]
    dataset.map(encode_example)
    # df=pd.DataFrame()
    # df['token_tensors']=tok_tensors
    # df['segment_tensor']=seg_tensors
    # df['labels']=lab_tensors
    return tok_tensors,seg_tensors,lab_tensors,pad_msk_tensors

In [18]:
tr_token_tensor,tr_segment_tensors,tr_label_tensors,tr_pad_msk_tensors=iter_df(train_dataset)
ts_token_tensor,ts_segment_tensors,ts_label_tensorsts,ts_pad_msk_tensors=iter_df(test_dataset)

  0%|          | 0/11660 [00:00<?, ?ex/s]

  0%|          | 0/1670 [00:00<?, ?ex/s]

In [19]:
from torch.utils.data import DataLoader,Dataset
class PhelixDataset(Dataset):
    def __init__(self, token_tens,label_tens,seg_tens,pd_msk_tens):
        #super().__init__()
        self.token_tens = token_tens
        self.seg_tens=seg_tens
        self.label_tens=label_tens
        self.pd_msk_tens=pd_msk_tens

    def __len__(self):
        return len(self.token_tens)
    
    def __getitem__(self, index):       
        tokens_id = self.token_tens[index]
        segment_id = self.seg_tens[index]
        label=self.label_tens[index]
        pad_mask=self.pd_msk_tens[index]

        return tokens_id,label,segment_id,pad_mask

In [20]:
batch_size=1
train_data = PhelixDataset(tr_token_tensor,tr_label_tensors,tr_segment_tensors,tr_pad_msk_tensors)
test_data = PhelixDataset(ts_token_tensor,ts_label_tensorsts,ts_segment_tensors,ts_pad_msk_tensors)
train_dataloader = DataLoader(train_data, batch_size = batch_size,shuffle=True,num_workers=3,pin_memory=True)
test_dataloader = DataLoader(test_data, batch_size = batch_size,shuffle=False,num_workers=3,pin_memory=True)
sample = next(iter(train_dataloader))
print(sample)

[tensor([[  101,  1022,  1023, 25682,  1023,  5354,  7610,  2013,  7987,  3593,
         26580,  2966,  2000,  4601,  2575,  2620, 12740, 21619,  2692,  2575,
          3931, 25604,  1997,  4002,  2475,  7987,  3593, 26580,  2966,  3328,
          2378,  9349, 17222,  2692, 13745,  3927,  3131,  8148, 18603,  2006,
          2710, 23290,  2860,  2475,  2015,  2620,  6904,  2595,  3104,  7123,
          2000,  3010,  4315, 18900,  6779,  2803,  3042,  4601,  2575, 18827,
         18139, 24434,  2581,  6904,  2595,  4601,  2575,  2620, 12740, 21619,
          2692,  2575,  3058, 25682,  4887,  2290,  2692,  2683,  2013,  2852,
          2745,  8825,  8915,  4221, 20850,  4887,  2213,  3042,  4601, 21084,
          2683,  2683, 28154, 21084,  6904,  2595,  4601,  2575,  4749, 16068,
          2620, 23499,  2193,  3931,  2164,  3104,  1016, 18777,  2023,  4807,
          3832,  3265,  5145,  8280,  5500, 15826, 21362,  3087,  2842,  1996,
          6254,  4807,  2089,  5383,  3167, 18777, 

In [21]:
for token, label,seg,pad_mask in train_dataloader:
    print(token.shape)
    print(label.shape)
    print(seg.shape)
    print(pad_mask.shape)
    break

torch.Size([1, 512])
torch.Size([1])
torch.Size([1, 512])
torch.Size([1, 512])


In [22]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')
model.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [4]:
saved_data='/media/umar_visionx/Backup Plus/Active/Faizan/textrect_saved_vectors'

In [24]:
import pickle
def embed(data_loader,train):
  #creating word embeddings
  model.eval()
  # token_labels_list=[]
  # concat_token_vect=[]#define list to append concatinated token vectors
  # doc_vectors_list=[]
  # doc_label_list=[]
  #to save data as arrays 
  token_labels_list_arr=[]
  concat_token_vect_arr=[]
  # doc_vectors_list_arr=[]
  # doc_label_list_arr=[]



  if train==True:
            ft=open(os.path.join(saved_data,'train_feat.pickle'), 'ab')
            lb=open(os.path.join(saved_data,'train_label.pickle'), 'ab')
  else:
            ft=open(os.path.join(saved_data,'test_feat.pickle'), 'ab')
            lb=open(os.path.join(saved_data,'test_label.pickle'), 'ab')
  for tokens,label,seg_id,_ in tqdm(data_loader):

      #moving to device
      label=int(label)
      tokens=tokens.to(device)
      seg_id=seg_id.to(device)
      with torch.no_grad():
        output = model(tokens,seg_id)
      #we set output_hidden_state=True in our pre trained model so, we will get the hidden states from all 13 layers at third positiob
      hidden_states = output[2]
      #For futher info please check https://mccormickml.com/2019/05/14/BERT-word-embeddings-tutorial/
      # Concatenating tensors for all layers and creating new dim with the help of stack
      token_embed = torch.stack(hidden_states, dim=0)#token embeddings(torch.Size([13, 1, 512, 768]))
      token_embed = torch.squeeze(token_embed, dim=1)#removing batch dimension(torch.Size([13, 512, 768]))
      token_embed = token_embed.permute(1,0,2)# swapping dimensions(torch.Size([512, 13, 768]))
      #iterating for each token in whole document/sentense
      to_append_vect=[]
      to_append_vect_arr=[]
      #to_append_label=[]
      for token in token_embed:
          #token vector of size 13*768 where 13 are number of layers while we want to get only last 4 layers i.e 4*768
          #after concatination last four layers vector dimensions will be 4*768=3,072(singlre word vector length i.e per token)
          #concatinating vectors
          add1 = torch.add(token[-1], token[-2])
          add2=torch.add(token[-3], token[-4])
          full_vector=torch.add(add1,add2)
          #concatcat_vector = torch.cat((token[-1], token[-2], token[-3], token[-4]), dim=0)
          # concat_token_vect.append(full_vector)
          #we have append 512 vectors of size 768 for each document
          # token_labels_list.append(label)
          token_labels_list_arr.append(label)
          concat_token_vect_arr.append(full_vector.cpu().numpy())

          to_append_vect_arr.append(full_vector.cpu().numpy())
          to_append_vect.append(full_vector)
          #to_append_label.append(label)
      # print(len(to_append_vect_arr))
      # print(len(to_append_vect))


      # doc_vectors_list.append(to_append_vect)
      # doc_label_list.append(label)
      # doc_vectors_list_arr.append(to_append_vect_arr)
      # doc_label_list_arr.append(label)

      # with open(os.path.join(saved_data,"train_features.pickle"),'ab') as f:
      #   pickle.dump(to_append_vect_arr,f)
      # with open(os.path.join(saved_data,"train_labels.pickle"),'ab') as f:
      #   pickle.dump(label,f)
      pickle.dump(to_append_vect_arr,ft)
      pickle.dump(label,lb)
  ft.close()
  lb.close()

  #return doc_vectors_list_arr,doc_label_list_arr

In [25]:
#Generating Embeddings

embed(train_dataloader,True)
embed(test_dataloader,False)

100%|██████████| 1670/1670 [01:35<00:00, 17.56it/s]


In [5]:
import pickle
#loading data from pickle file
def ext_data(filepath):

    data = []
    with open(filepath, 'rb') as fr:
        try:
            while True:
                data.append(pickle.load(fr))
        except EOFError:
            pass
    return data

In [6]:
from tqdm import tqdm
train_feat_path='/media/umar_visionx/Backup Plus/Active/Faizan/textrect_saved_vectors/train_feat.pickle'
train_label_path='/media/umar_visionx/Backup Plus/Active/Faizan/textrect_saved_vectors/train_label.pickle'
test_feat_path='/media/umar_visionx/Backup Plus/Active/Faizan/textrect_saved_vectors/test_feat.pickle'
test_label_path='/media/umar_visionx/Backup Plus/Active/Faizan/textrect_saved_vectors/test_label.pickle'
doc_vectors_list_arr=ext_data(train_feat_path)
doc_label_list_arr=ext_data(train_label_path)
test_vector_list_arr=ext_data(test_feat_path)
test_label_list_arr=ext_data(test_label_path)

In [7]:
# del doc_vectors_list_arr
# del doc_label_list_arr
# del test_vector_list_arr
# del test_label_list_arr

In [8]:
#this list have document vectors,every vector,every document have 512 token vectors and every oken vector have 3072 entries
print(len(doc_vectors_list_arr))
print(len(doc_label_list_arr))
print(len(doc_vectors_list_arr[1]))
print(len(doc_label_list_arr))
print(len(doc_vectors_list_arr[1][1]))

11660
11660
512
11660
768


In [9]:
#splitiing the data into train and validation
x_t, x_v, y_t, y_v = train_test_split(doc_vectors_list_arr, doc_label_list_arr, test_size=0.1,random_state=42,stratify=doc_label_list_arr)
print(len(x_t))
print(len(x_v))

10494
1166


In [10]:
#converting features into numpy array
x_train=[]
y_train=[]
x_val=[]
y_val=[]
x_test=[]
y_test=[]

#train set
for ent in x_t:
    x_train.append(np.array(ent,dtype=np.float32))

for ent in y_t:
    y_train.append(np.array(ent,dtype=np.int32))

#validation set
for ent in x_v:
    x_val.append(np.array(ent,dtype=np.float32))

for ent in y_v:
     y_val.append(np.array(ent,dtype=np.int32))

#test set
for ent in test_vector_list_arr:
    x_test.append(np.array(ent,dtype=np.float32))

for ent in test_label_list_arr:
    y_test.append(np.array(ent,dtype=np.int32))

print(len(x_train))
print(len(y_train))
print(len(x_val))
print(len(y_val))
print(len(x_test))
print(len(y_test))

    

10494
10494
1166
1166
1670
1670


In [10]:
# del x_train
# del y_train
# del x_val
# del y_val

In [ ]:
class AttentionLayer(Layer):
    """
    Source: https://www.cs.cmu.edu/~hovy/papers/16HLT-hierarchical-attention-networks.pdf
            https://humboldt-wi.github.io/blog/research/information_systems_1819/group5_han/
   
    """


    def __init__(self,attention_dim=768,return_coefficients=False,**kwargs):
        # Initializer 
        self.supports_masking = True
        self.return_coefficients = return_coefficients
        self.init = initializers.get('glorot_uniform') # initializes values with uniform distribution
        self.attention_dim = attention_dim
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # print(input_shape[0])
        # print(input_shape[-1])
        # print(input_shape[1])
        # Builds all weights
        # W = Weight matrix, b = bias vector, u = context vector
        assert len(input_shape) == 3
        self.W = K.variable(self.init((int(input_shape[-1]), self.attention_dim)),name='W')
        self.b = K.variable(self.init((self.attention_dim, )),name='b')
        self.u = K.variable(self.init((self.attention_dim, 1)),name='u')
        self._trainable_weights = [self.W, self.b, self.u]

        super(AttentionLayer, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, hit, mask=None):
        # Here, the actual calculation is done
        uit = K.bias_add(K.dot(hit, self.W),self.b)
        uit = K.tanh(uit)
        
        ait = K.dot(uit, self.u)
        ait = K.squeeze(ait, -1)
        ait = K.exp(ait)
        
        if mask is not None:
            ait *= K.cast(mask, K.floatx())

        ait /= K.cast(K.sum(ait, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        ait = K.expand_dims(ait)
        weighted_input = hit * ait
        
        if self.return_coefficients:
            return [K.sum(weighted_input, axis=1), ait]
        else:
            return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        if self.return_coefficients:
            return [(input_shape[0], int(input_shape[-1])), (input_shape[0], int(input_shape[-1]), 1)]
        else:
            return input_shape[0], int(input_shape[-1])

    # def get_config(self):

    #     config = super().get_config().copy()
    #     config.update({
    #         'attention_dim ': self.attention_dim ,
    #         'return_coefficients': self.return_coefficients,
    #     })
    #     return config

    def get_config(self):#to avoiding saving problem.check(https://stackoverflow.com/questions/50837728/valueerror-unknown-layer-capsulelayer and    
        #https://stackoverflow.com/questions/58678836/notimplementederror-layers-with-arguments-in-init-must-override-get-conf)
        # For serialization with 'custom_objects'
        config = super().get_config()
        config['attention_dim'] = self.attention_dim
        config['return_coefficients'] = self.return_coefficients
        return config

In [24]:
import tensorflow
import tensorflow as tf
# embedding_matrix = np.random.random((VOCAB_SIZE, EMBEDDING_DIM))
# max_sentenses=1
# embedding_layer = Embedding(VOCAB_SIZE,EMBEDDING_DIM,input_length=MAX_SEQUENCE_LENGTH,trainable=False,weights=[embedding_matrix],name='word_embedding')
MAX_SEQUENCE_LENGTH=512
EMBEDDING_DIM=768
max_sentenses=1

# Words  attention
emb_input = Input(shape=(MAX_SEQUENCE_LENGTH,EMBEDDING_DIM), dtype='float32',name='embeddings_input')
# #word_sequences = embedding_layer(word_input)
gru_word = Bidirectional(GRU(768, return_sequences=True),name='word_GRU')(emb_input)#by default if we set treturn_sequence=False it will
#return only last words hidden state is returm only as the output having vector dimensions=2*GRU units 
#for bidirectional
# word_dense = Dense(1536, activation='relu', name='word_dense')(gru_out) 
word_attention,word_coff= AttentionLayer(EMBEDDING_DIM,return_coefficients=True,name='word_attention')(gru_word)

# dense_out = Dense(512,activation='relu', name='word_dense')(word_attention)#we may use different number of units
# preds = Dense(13, activation='softmax',name='output')(dense_out)
# model = Model(emb_input, preds)
WordEnc = Model(inputs = emb_input,outputs = word_attention)#word encoder



#Sentence Attention
sentence_inp=Input(shape=(max_sentenses, MAX_SEQUENCE_LENGTH,EMBEDDING_DIM), dtype='float32')
sent_encoder = TimeDistributed(WordEnc,name='sent_encd')(sentence_inp)
gru_sentense = Bidirectional(GRU(100, return_sequences=True),name='sentense_GRU')(sent_encoder)
sentense_attention,sentense_coff= AttentionLayer(EMBEDDING_DIM,True,name='sentense_attention')(gru_sentense)
dense_out = Dense(512,activation='relu', name='sentense_dense')(sentense_attention)#we may use different number of units
dropout = Dropout(0.4,name='dropout')(dense_out)
preds = Dense(13, activation='softmax',name='output')(dropout)
model = Model(sentence_inp, preds)

In [25]:

model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
print(WordEnc.summary())
print(model.summary())


Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embeddings_input (InputLaye  [(None, 512, 768)]       0         
 r)                                                              
                                                                 
 word_GRU (Bidirectional)    (None, 512, 1536)         7087104   
                                                                 
 word_attention (AttentionLa  [(None, 1536),           1181184   
 yer)                         (None, 512, 1)]                    
                                                                 
Total params: 8,268,288
Trainable params: 8,268,288
Non-trainable params: 0
_________________________________________________________________
None
Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)      

In [11]:
#Adding dimension into training and test data and converting into numpy array
# x_tr = np.array([x_train])
# y_tr = np.array(y_train)
x_ts=np.array([x_test])
y_ts=np.array(y_test)
#Reshaping inputs
# print(x_tr.shape)
# print(y_tr.shape)
print(x_ts.shape)
print(y_ts.shape)
# x_trn=np.transpose(x_tr, (1, 0, 2,3))
x_tst=np.transpose(x_ts, (1, 0, 2,3))
# print(x_trn.shape)
print(x_tst.shape)

(1, 1670, 512, 768)
(1670,)
(1670, 1, 512, 768)


In [12]:
#data loader
from tensorflow.keras.utils import Sequence
import math
class DataGenerator(Sequence):

    def __init__(self, x_train, y_train,batch_size):
        self.x, self.y = x_train, y_train
        self.batch_size = batch_size

    def __len__(self):
        return math.ceil(len(self.x) / self.batch_size)#ValueError: axes don't match array

    def __getitem__(self, idx):
        
        x_tr = np.array([self.x[idx * self.batch_size:(idx + 1) *
        self.batch_size]])#we also added an extra dimension according to our requirement
        y_tr = np.array(self.y[idx * self.batch_size:(idx + 1) *
        self.batch_size])

        x_trn=np.transpose(x_tr, (1, 0, 2,3))#reshape the input

        return x_trn,y_tr

In [13]:
train_generator=DataGenerator(x_train, y_train,32)
val_generator=DataGenerator(x_val, y_val,32)
#test_generator = DataGenerator(x_test,y_test,32)

In [14]:
#checking the total samples in each class(label)
from collections import Counter
print(f"labels: {Counter(doc_label_list_arr).keys()}")
print(f"samples in each label: {Counter(doc_label_list_arr).values()}")

labels: dict_keys([4, 3, 2, 12, 7, 8, 1, 5, 11, 0, 9, 6, 10])
samples in each label: dict_values([491, 1542, 1438, 1312, 780, 982, 1045, 969, 2363, 588, 72, 23, 55])


In [15]:
#making class weights dictionary (our classes are imbalance)
#getting class having maximum samples
class_samples={}
train_text='/media/umar_visionx/Backup Plus/Active/Faizan/textrect_dataset'
for folder in os.listdir(train_text):
    folder_path=os.path.join(train_text,folder)
    class_samples[folder]=len(os.listdir(folder_path))
print(f'max samples in class ( {max(class_samples, key=class_samples.get)}={class_samples[max(class_samples, key=class_samples.get)]})')
max_samples=int(class_samples[max(class_samples, key=class_samples.get)])
print('***********')
#getting class weights
weights={}
for folder in os.listdir(train_text):
    folder_path=os.path.join(train_text,folder)
    weights[folder]=round(max_samples/int(len(os.listdir(folder_path))),2)
print(f'class weights: {weights}')
print("********")
#changing the names of keys in class weight dictionary
class_weight={}
for (keys1,values1),(keys2,values2) in zip(weights.items(),label_index.items()):

    class_weight[int(values2)]=values1

print(f'class weights: {class_weight}')

max samples in class ( Referral Letter=2363)
***********
class weights: {'Admin Note': 4.02, 'Clinical History And Summary': 2.26, 'Consult Note': 1.64, 'Cover Page': 1.53, 'Imaging Note': 4.81, 'Insurance Authorization': 2.44, 'Intake Forms': 102.74, 'Lab Test': 3.03, 'Other': 2.41, 'Patient Profile': 32.82, 'Prescriptions': 42.96, 'Referral Letter': 1.0, 'Requisition Form': 1.8}
********
class weights: {0: 4.02, 1: 2.26, 2: 1.64, 3: 1.53, 4: 4.81, 5: 2.44, 6: 102.74, 7: 3.03, 8: 2.41, 9: 32.82, 10: 42.96, 11: 1.0, 12: 1.8}


In [21]:

#generator=training_generator
#from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
import warnings
warnings.filterwarnings('ignore')
#batch_size = 32
checkpoint_path='/media/umar_visionx/Backup Plus/Active/Faizan/weighted_textrect_model/'#/my_model.h5
#es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)#early stopping
#mc = ModelCheckpoint(checkpoint_path, monitor='val_acc', mode='max', verbose=1, save_best_only=True)#model checkpoint
loss_list=[]
#loss_list.append(history.history['val_loss'][0])
history=model.fit(train_generator,validation_data=val_generator, epochs=1,class_weight=class_weight,shuffle=True)
loss_list.append(history.history['val_loss'][0])
for i in range(9):
    history=model.fit(train_generator,validation_data=val_generator, epochs=1,class_weight=class_weight,shuffle=True)
    loss=history.history['val_loss'][0]
    if loss<loss_list[-1]:
        #saving the model
        model.save(checkpoint_path)
        loss_list.append(history.history['val_loss'][0])

 94/328 [=======>......................] - ETA: 17:22 - loss: 0.6597 - acc: 0.9013

In [3]:
checkpoint_path='/media/umar_visionx/Backup Plus/Active/Faizan/weighted_textrect_model/'#/my_model.h5
model = tf.keras.models.load_model('/media/umar_visionx/Backup Plus/Active/Faizan/model_keras/')
model.summary()

2022-04-05 12:05:06.468034: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-05 12:05:06.468175: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-04-05 12:05:06.468268: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-04-05 12:05:06.468359: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2022-04-05 12:05:06.468446: W tensorflow/stream_executor/platform/default/dso_loader.cc:64

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1, 512, 768)]     0         
                                                                 
 sent_encd (TimeDistributed)  (None, 1, 1536)          8268288   
                                                                 
 sentense_GRU (Bidirectional  (None, 1, 200)           982800    
 )                                                               
                                                                 
 sentense_attention (Attenti  [(None, 200),            155136    
 onLayer)                     (None, 1, 1)]                      
                                                                 
 sentense_dense (Dense)      (None, 512)               102912    
                                                                 
 dropout (Dropout)           (None, 512)               0   

In [17]:
history.history

{'loss': [0.14990822970867157],
 'acc': [0.9527348875999451],
 'val_loss': [0.24877971410751343],
 'val_acc': [0.9288164377212524]}

In [17]:
score = model.evaluate(x_tst, y_ts, verbose = 1) 

print('Test loss:', score[0]) 
print('Test accuracy:', score[1])

2022-04-03 11:57:52.176247: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 2626682880 exceeds 10% of free system memory.
2022-04-03 11:57:57.871257: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 50331648 exceeds 10% of free system memory.
2022-04-03 11:57:57.871363: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 50331648 exceeds 10% of free system memory.
2022-04-03 11:57:57.871482: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 50331648 exceeds 10% of free system memory.
2022-04-03 11:57:57.871693: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 50331648 exceeds 10% of free system memory.


53/53 [==============================] - 81s 1s/step - loss: 1.4804 - acc: 0.7204
Test loss: 1.4804365634918213
Test accuracy: 0.7203592658042908


In [18]:
predictions=model.predict(x_tst)
pred = np.argmax(predictions, axis = 1)
print(pred) 

[ 0  0  0 ... 12 12 12]


In [19]:
from sklearn.metrics import classification_report
print(classification_report(y_ts,pred))

              precision    recall  f1-score   support

           0       0.78      0.64      0.71        76
           1       0.68      0.72      0.70       336
           2       0.67      0.77      0.72       349
           3       0.93      0.96      0.95       172
           4       0.65      0.82      0.73        45
           5       0.47      0.26      0.33        31
           6       0.00      0.00      0.00         8
           7       0.75      0.89      0.82       130
           8       0.75      0.34      0.46       247
           9       0.73      0.66      0.69        41
          10       0.40      0.25      0.31         8
          11       0.63      0.95      0.75       154
          12       1.00      0.84      0.91        73

    accuracy                           0.72      1670
   macro avg       0.65      0.62      0.62      1670
weighted avg       0.73      0.72      0.71      1670



/home/umar_visionx/anaconda3/envs/lmv2/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/umar_visionx/anaconda3/envs/lmv2/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/umar_visionx/anaconda3/envs/lmv2/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_sta

In [20]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,pred))

0.7203592814371258


In [4]:
#feature extraction
predictions=model.predict(x_tst)
labels=y_ts
print(preictions[1].shape)
print(labels[1].shape)

NameError: name 'x_tst' is not defined